In [1]:
import pandas as pd
import plotly.graph_objects as go
import ipywidgets as widgets
from IPython.display import display, clear_output

# ── 1) Load your DataFrame ─────────────────────────────────────────────────────
df = pd.read_csv('cynsma.csv', parse_dates=['Start','End'])
df.columns = df.columns.str.strip()

# Coerce any bad End-dates to NaT, then build a string version that is empty when NaT
df['End'] = pd.to_datetime(df['End'], errors='coerce')
df['End_str'] = df['End'].dt.strftime('%Y-%m-%d').fillna('')

# ── 2) Build the widgets ────────────────────────────────────────────────────────
cusip_input     = widgets.Text(description='CUSIP filter:', placeholder='e.g. 09711A2C1')
tenor_slider    = widgets.FloatRangeSlider(
    description="Tenor (mo's):",
    min=0, max=df["Tenor(mo's)"].max(),
    step=0.1,
    value=(0, df["Tenor(mo's)"].max()),
    continuous_update=False
)
status_dropdown = widgets.Dropdown(
    options=['All'] + sorted(df['Status'].unique()),
    description='Status:'
)
toggle_avg_btn  = widgets.ToggleButton(
    description='Show Average',
    value=False,
    button_style='info'
)

out = widgets.Output()

# ── 3) The plot-update function ─────────────────────────────────────────────────
def update_plot(_=None):
    with out:
        clear_output(wait=True)
        print("Loading…")
    # apply filters
    d = df.copy()
    if cusip_input.value:
        d = d[d['CUSIP'].str.contains(cusip_input.value, case=False)]
    lo, hi = tenor_slider.value
    d = d[d["Tenor(mo's)"].between(lo, hi)]
    if status_dropdown.value != 'All':
        d = d[d['Status'] == status_dropdown.value]

    # build the figure
    fig = go.Figure()

    # scatter points
    fig.add_trace(go.Scatter(
        x=d['Start'],
        y=d["Tenor(mo's)"],
        mode='markers',
        marker=dict(
            size=8,
            color=d['Status'].map({'Alive':'lightgreen','Called':'crimson'}),
            line=dict(width=1, color='white')
        ),
        customdata=d[['CUSIP','Start','End_str','Status',"Tenor(mo's)",'Yield','Issuer']],
        hovertemplate=(
            "CUSIP: %{customdata[0]}<br>"
            "Start: %{customdata[1]|%Y-%m-%d}<br>"
            "End: %{customdata[2]}<br>"
            "Status: %{customdata[3]}<br>"
            "Tenor: %{customdata[4]} mo<br>"
            "Yield: %{customdata[5]}<br>"
            "Issuer: %{customdata[6]}<extra></extra>"
        )
    ))

    # optional average-tenor line
    if toggle_avg_btn.value and not d.empty:
        avg_tenor = d["Tenor(mo's)"].mean()
        fig.add_hline(
            y=avg_tenor,
            line_dash='dash',
            line_color='gold',
            annotation_text=f"Avg Tenor: {avg_tenor:.2f} mo",
            annotation_position="top left"
        )

    fig.update_layout(
        title="Autocallables: Tenor vs. Start Date",
        template="plotly_dark",
        xaxis_title="Start Date",
        yaxis_title="Tenor (months)",
        margin=dict(l=40, r=20, t=50, b=40),
        height=450,
        width=700
    )

    with out:
        clear_output()
        display(fig)

# ── 4) Wire up events & do initial draw ─────────────────────────────────────────
for w in (cusip_input, tenor_slider, status_dropdown, toggle_avg_btn):
    w.observe(update_plot, names='value')

update_plot()

# ── 5) Layout ─────────────────────────────────────────────────────────────────
controls = widgets.HBox([cusip_input, status_dropdown, toggle_avg_btn])
ui = widgets.VBox([controls, tenor_slider, out])
display(ui)
